In [ ]:
import tensorflow as tf
import numpy as np
#training phase
num_features = 10 
input_shape = (num_features,)


input_data = tf.keras.Input(shape=input_shape)

hidden_layer = tf.keras.layers.Dense(units=64, activation=tf.nn.relu)(input_data)
output_layer = tf.keras.layers.Dense(units=num_features, activation=None)(hidden_layer)

model = tf.keras.Model(inputs=input_data, outputs=output_layer)

model.compile(optimizer='adam', loss='mean_squared_error')

num_samples = 1000
input_data = np.random.randn(num_samples, num_features)
output_data = input_data  

model.fit(input_data, output_data, epochs=100, batch_size=64)

#new data inserted here (testing phase)
